In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot  as plt
import os


In [ ]:
print(xgb.__version__)

In [ ]:
# dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime']
dateTimeCols=['open_datetime','close_datetime']

cateFeatures=['product_type','brand','incident_type','service_type','is_failure_type','sla']

# numericFeatures=['count_detail','open_to_close_hour','response_to_resolved_hour','open_to_response_hour','resolved_to_close_hour']
numericFeatures=['count_detail','open_to_close_hour']

colLabel='severity_label'


calLabelRefInfo=[colLabel,'severity_name']

colFeatures=cateFeatures+numericFeatures

cols=[colLabel]+colFeatures

print(cols)

from_date='2023-01-10'

# Load and Prepare Data

In [ ]:
df_incident = pd.read_csv('incident_data.csv',parse_dates=dateTimeCols)
print(f"Incident From {df_incident['open_datetime'].min()} To {df_incident['open_datetime'].max()}")
df_incident=df_incident.query("open_datetime>=@from_date")

df_incident=df_incident[cols]
print(df_incident.info())
df_incident.tail()

# Process features and target class

In [ ]:
# cateFeatures_cateLabel=cateFeatures.copy()+[colLabel]
print(f"{cateFeatures} and {colLabel}")
X = df_incident.loc[:,colFeatures]
X[cateFeatures]=X[cateFeatures].astype("category") 

print(X.info())
X.sample(10)


In [ ]:
y=df_incident.loc[:,colLabel]
print(y.info())
y.sample(10)
# y=y.astype("category")

In [ ]:
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# print(label_encoder.classes_)

# print(y[-5:])

# Load Model Model to Predict

In [ ]:
model_xg = xgb.Booster()  # init model
model_xg.load_model('model.bst')  # load data

In [ ]:
dtest = xgb.DMatrix(data=X,label=y,enable_categorical=True)
y_pred_values = model_xg.predict(dtest)
yPredictionsList = [round(value) for value in y_pred_values]

accuracy_xg = accuracy_score(y, yPredictionsList)
print("XGBoost Accuracy: %.2f%%" % (accuracy_xg * 100.0))

In [ ]:
yPrediction=pd.Series(yPredictionsList).to_frame("severity_prediction")

In [ ]:
dfAll=pd.concat([yPrediction,y,X],axis=1)
dfAll.info()
dfAll